In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [4]:
import json
with open('/Users/danielbarella/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [5]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Pizza'

In [6]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NY_pizza.json"
JSON_FILE

'Data/results_in_progress_NY_pizza.json'

In [7]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to file.


In [8]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [9]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()


dict_keys(['businesses', 'total', 'region'])

In [10]:
## How many results total?
total_results = results['total']
total_results

16400

In [11]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [12]:
# Import additional packages for controlling our loop
#import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

820

In [13]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [14]:
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/820 [00:00<?, ?it/s]

In [15]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/820 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

In [16]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [17]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [18]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


820

In [19]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/820 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [20]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/I4gm7i...,False,https://www.yelp.com/biz/prince-street-pizza-n...,4745,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,2209.311618
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/NVoLFl...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2601,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,1289.857286
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media4.fl.yelpcdn.com/bphoto/LuSzR8...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,2903,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,2268.491950
3,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/EBwiiS...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,864,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,1868.295512
4,WIhm0W9197f_rRtDziq5qQ,lombardis-pizza-new-york-4,Lombardi's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/UZ6V_h...,False,https://www.yelp.com/biz/lombardis-pizza-new-y...,6523,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7215934960083, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '32 Spring St', 'address2': '', '...",+12129417994,(212) 941-7994,2133.973859


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,sYRhc9liDE2tzMyS-pVwhg,sunny-john-pizzeria-brooklyn,Sunny John Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/9ZMSG4...,False,https://www.yelp.com/biz/sunny-john-pizzeria-b...,41,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.6190419, 'longitude': -73.9226...","[delivery, pickup]",$,"{'address1': '5408 Ave N', 'address2': '', 'ad...",+17182416115,(718) 241-6115,10878.347197
996,1PBlkYAEFhc0cpqNNvlkWg,the-ear-inn-new-york,The Ear Inn,https://s3-media2.fl.yelpcdn.com/bphoto/W2D6yu...,False,https://www.yelp.com/biz/the-ear-inn-new-york?...,425,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 40.725925, 'longitude': -74.009489}","[delivery, pickup]",$$,"{'address1': '326 Spring St', 'address2': '', ...",+12122269060,(212) 226-9060,3310.724707
997,XyVI14tZJsTQHMJ5Wva1JQ,johnny-napkins-union-3,Johnny Napkins,https://s3-media2.fl.yelpcdn.com/bphoto/2WsKe4...,False,https://www.yelp.com/biz/johnny-napkins-union-...,235,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.6890734, 'longitude': -74.2468...","[delivery, pickup]",$,"{'address1': '1424 Morris Ave', 'address2': ''...",+19086887500,(908) 688-7500,22656.171082
998,IirHsL3-vtnxkKrhFLBZWA,bobs-thin-crust-pizza-tappan,Bob's Thin Crust Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/H_4q2l...,False,https://www.yelp.com/biz/bobs-thin-crust-pizza...,90,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 41.0224699, 'longitude': -73.9515...","[delivery, pickup]",$,"{'address1': '83 Old Tappan Rd', 'address2': '...",+18453983034,(845) 398-3034,35160.156448
999,Lgsyd4pWz_aibPj2CoPCqw,a-legna-morristown,A Legna,https://s3-media2.fl.yelpcdn.com/bphoto/5Ocn6C...,False,https://www.yelp.com/biz/a-legna-morristown?ad...,32,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.79645020103329, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '47 S Park Pl', 'address2': '', '...",+19739980415,(973) 998-0415,43382.275218


In [21]:
# check for duplicate results
final_df.duplicated().sum()

TypeError: unhashable type: 'list'

In [22]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

179

In [23]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [ ]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)